In [ ]:
import pandas as pd
import json
import xml.etree.ElementTree as ET

# Load and process CSV data
def process_csv(file_path):
    csv_data = pd.read_csv(file_path)
    csv_data = csv_data.rename(columns=str.lower)  # Convert column names to lowercase
    csv_data = csv_data.rename(columns={
        'first name': 'firstname',
        'second name': 'lastname',
        'age (years)': 'age',
        # Rename other columns if necessary
    })
    csv_data['age'] = csv_data['age'].astype(str)  # Standardize 'age' column to string type
    return csv_data

# Load and process JSON data
def process_json(file_path):
    with open(file_path, 'r') as file:
        json_data = json.load(file)
    df = pd.json_normalize(json_data)
    df = df.rename(columns=str.lower)  # Convert column names to lowercase
    df['age'] = df['age'].astype(str)  # Standardize 'age' column to string type
    return df

# Load and process XML data
def process_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = [user.attrib for user in root.findall('user')]
    df = pd.DataFrame(data)
    df = df.rename(columns=str.lower)  # Convert column names to lowercase
    df['age'] = df['age'].astype(str)  # Standardize 'age' column to string type
    return df

# Define the file paths
csv_path = "C:\\Users\\mlawal\\Downloads\\Dissertation Papers\\cetm50_23_4_data (2)\\user_data_23_4.csv"
json_path = "C:\\Users\\mlawal\\Downloads\\Dissertation Papers\\cetm50_23_4_data (2)\\user_data_23_4.json"
xml_path = "C:\\Users\\mlawal\\Downloads\\Dissertation Papers\\cetm50_23_4_data (2)\\user_data_23_4.xml"


# Process the files
csv_data = process_csv(csv_path)
json_data = process_json(json_path)
xml_data = process_xml(xml_path)

# Merge the data on 'firstname' and 'lastname' columns
merged_data = pd.merge(csv_data, json_data, on=['firstname', 'lastname'], how='outer', suffixes=('_csv', '_json'))
merged_data = pd.merge(merged_data, xml_data, on=['firstname', 'lastname'], how='outer')

# Display the first few rows of the merged DataFrame
print(merged_data.head())

In [ ]:
# Create a new 'consolidated_age' column
merged_data['consolidated_age'] = merged_data['age_csv']
merged_data['consolidated_age'] = merged_data['consolidated_age'].combine_first(merged_data['age_json'])
merged_data['consolidated_age'] = merged_data['consolidated_age'].combine_first(merged_data['age'])

# Drop the original age columns
merged_data.drop(columns=['age_csv', 'age_json', 'age'], inplace=True)

# Check the updated DataFrame
print(merged_data.head())


In [ ]:
# Create a new 'consolidated_sex' column
merged_data['consolidated_sex'] = merged_data['sex_x'].combine_first(merged_data['sex_y'])

# Drop the original sex columns
merged_data.drop(columns=['sex_x', 'sex_y'], inplace=True)

# Check the updated DataFrame
print(merged_data.head())


In [ ]:
print(merged_data.columns)

In [ ]:
# Create a new 'consolidated_address_postcode' column
merged_data['consolidated_address_postcode'] = merged_data['address_postcode_x'].combine_first(merged_data['address_postcode_y'])

# Drop the original address_postcode columns
merged_data.drop(columns=['address_postcode_x', 'address_postcode_y'], inplace=True)

# Check the updated DataFrame
print(merged_data.head())


In [ ]:
# Get the list of all column names
columns = list(merged_data.columns)

# Remove 'consolidated_age' and 'consolidated_sex' from the list
columns.remove('consolidated_age')
columns.remove('consolidated_sex')

# Insert 'consolidated_age' and 'consolidated_sex' right after 'lastname'
lastname_index = columns.index('lastname')
columns.insert(lastname_index + 1, 'consolidated_age')
columns.insert(lastname_index + 2, 'consolidated_sex')

# Reorder the DataFrame columns
merged_data = merged_data[columns]

# Check the updated DataFrame
print(merged_data.head())


In [ ]:
# Rename the columns by removing the 'consolidated' prefix
merged_data.rename(columns={
    'consolidated_age': 'age',
    'consolidated_sex': 'sex',
    'consolidated_address_postcode': 'address_postcode'
}, inplace=True)

# Check the updated DataFrame
print(merged_data.head())


In [ ]:
print(merged_data.columns)

In [ ]:
import mysql.connector

student_id = 'bi63as'

config = {
    'user': f'student_{student_id}',
    'password': 'iE93F2@8EhM@1zhD&u9M@K',
    'host': 'europa.ashley.work',
    'database': f'student_{student_id}',
}

try:
    # Establish a database connection
    cnx = mysql.connector.connect(**config)
    print("Connection successfully established!")


finally:
    # Close the connection
    if cnx.is_connected():
        cnx.close()
        print("Connection closed.")

In [ ]:
import pandas as pd
import mysql.connector

# Assuming 'merged_data' is your DataFrame containing the merged data
# merged_data = ...

# Database configuration
student_id = 'bi63as'
config = {
    'user': f'student_{student_id}',
    'password': 'iE93F2@8EhM@1zhD&u9M@K',
    'host': 'europa.ashley.work',
    'database': f'student_{student_id}',
}

# Connect to the database
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()
# SQL query to insert data
insert_query = """
INSERT INTO merged_records (
    firstname, 
    lastname, 
    age,
    sex,
    vehicle_make, 
    vehicle_model, 
    vehicle_year, 
    vehicle_type, 
    iban, 
    credit_card_number, 
    credit_card_security_code, 
    credit_card_start_date, 
    credit_card_end_date, 
    address_main, 
    address_city, 
    debt, 
    debt_amount, 
    debt_time_period_years, 
    retired, 
    dependants, 
    marital_status, 
    salary, 
    pension, 
    company, 
    commute_distance, 
    address_postcode
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the DataFrame and insert each row
for i, row in merged_data.iterrows():
    try:
        # Convert all NaN values to None and create a tuple for insertion
        data_tuple = tuple([None if pd.isna(value) else value for value in row])
        cursor.execute(insert_query, data_tuple)
    except Exception as e:
        print(f"Error inserting row {i}: {e}")

# Commit the changes to the database
cnx.commit()
